<a href="https://colab.research.google.com/github/ranieri-unimi/malchiodi-2022/blob/main/ukraine.colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### run once

In [1]:
!pip install pyspark
!pip install findspark

In [2]:
import os
os.environ["KAGGLE_USERNAME"] = 'ranieriunimi'
os.environ["KAGGLE_KEY"] = str(hex(232307088475198570779809482024044346960))[2:]

In [3]:
ref = 'bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows'
!mkdir datasets
!kaggle datasets download $ref --unzip -p ./datasets

mkdir: cannot create directory ‘datasets’: File exists
 99% 1.82G/1.83G [00:12<00:00, 100MB/s] 
100% 1.83G/1.83G [00:12<00:00, 152MB/s]


## ukraine

In [4]:
SAMPLE_SIZE = 512

In [5]:
import numpy as np
import pandas as pd
import csv
import re
import string
import random

In [6]:
import pyspark
import findspark

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [8]:
nltk.download('all');

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

In [42]:
# https://www.pluralsight.com/guides/building-a-twitter-sentiment-analysis-in-python

def preprocess_tweet_text(tweet):
    index, tweet = tweet

    tweet.lower()

    # cleanings 🧹

    # urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)

    # @ and #
    tweet = re.sub(r'\@\w+|\#','', tweet)

    # punctuations
    # tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    tweet = tweet.translate(str.maketrans(string.punctuation+'…’', ' '*(len(string.punctuation)+2)))  # puntctuation to spaces


    tweet_tokens = word_tokenize(tweet)

    # emojitter
    wrds = [e for word in tweet_tokens for e in re.findall(r"(\w+|[^\w ]+)", word)]
    # TODO split also emoji-goups

    # stopwords
    filtered_words = [w for w in wrds if not w in set(stopwords.words('english'))]
    
    # stemmatize
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]

    # lemmatize
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
  
    return (index, lemma_words)

In [10]:
# load dataset 
filename = r"./datasets/UkraineCombinedTweetsDeduped20220227-131611.csv.gzip"
pd.set_option("display.max_columns", None)
df = pd.read_csv(filename, compression='gzip', index_col=0, encoding='utf-8', quoting=csv.QUOTE_ALL)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [11]:
#lang_hist = {l:df[df.language == l].size for l in df.language.unique()}

In [12]:
datalist = df[df.language == 'en'].text.tolist()

In [13]:
if SAMPLE_SIZE:
    datalist = random.sample(datalist, SAMPLE_SIZE)

### hadoooooop

In [14]:
# import findspark
# findspark.init("spark-3.1.1-bin-hadoop3.2") # SPARK_HOME
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [49]:
rdd = sc.parallelize(enumerate(datalist))

In [50]:
dataPipe = rdd.map(preprocess_tweet_text)

In [51]:
# indexing = dataset.flatMap(lambda _, v : [(e,e) for e in v]).reduceByKey(lambda k, v : k)

### alternative

In [52]:
# nen_rdd = sc.parallelize(datalist)

In [53]:
# elemListPipe = nen_rdd.flatMap(lambda word_list : word_list).distinct()
# elem_index = sc.parallelize(enumerate(elemListPipe.collect()))

### counts D1

In [114]:
def is_sub(sub, lst) : return all(e in lst for e in sub)
def add(a, b) : return a+b
def splat(t): return tuple(sorted(list(j for i in t for j in (i if isinstance(i, tuple) else (i,)))))
def doubled(t): return len(set(t)) == len(t)

In [104]:
PCENT = .005

In [116]:
countOnePipe = dataPipe.flatMap(lambda x: x[-1]).map(lambda x: (x,1)).reduceByKey(add)
countOnePipe.take(5)

[('FM', 1), ('go', 19), ('peopl', 63), ('even', 8), ('continu', 6)]

In [117]:
THRESHOLD = countOnePipe.map(lambda x : x[-1]).reduce(lambda a,b : a+b) * PCENT

In [118]:
# filter non frequent
frequentOnePipe = countOnePipe.filter(lambda x: x[-1] > THRESHOLD).map(lambda x: (1, x[0]))
frequentOnePipe.take(5)

[(1, 'peopl'),
 (1, 'world'),
 (1, 'nato'),
 (1, 'putin'),
 (1, 'russiaukrainewar')]

#### 2wo

In [119]:
# generate candidate pairs
candidateTwoPipe = frequentOnePipe.join(frequentOnePipe).map(lambda x : x[-1]).map(splat).distinct().filter(doubled)
candidateTwoPipe.take(5)

[('peopl', 'world'),
 ('peopl', 'putin'),
 ('peopl', 'russiaukrainewar'),
 ('peopl', 'ukrainian'),
 ('nato', 'world')]

In [120]:
# count pair frequency
candidateTwo = candidateTwoPipe.collect() # pair in MEM
countTwoPipe = dataPipe.map(lambda x : [(pair, is_sub(pair, x[-1])) for pair in candidateTwo] ).flatMap(lambda x : x).reduceByKey(add)
countTwoPipe.take(5)

[(('peopl', 'world'), 17),
 (('peopl', 'putin'), 7),
 (('peopl', 'russiaukrainewar'), 4),
 (('peopl', 'ukrainian'), 20),
 (('nato', 'world'), 4)]

In [74]:
#THRESHOLD = countTwoPipe.map(lambda x : x[-1]).reduce(lambda a,b : a+b) * PCENT

In [121]:
# filter non frequent
frequentTwoPipe = countTwoPipe.filter(lambda x: x[-1] > THRESHOLD).map(lambda x: (1, x[0]))
frequentTwoPipe.take(5)

[(1, ('russia', 'russian')),
 (1, ('the', 'ukrain')),
 (1, ('russian', 'ukrain')),
 (1, ('russia', 'ukrain')),
 (1, ('I', 'ukrain'))]

#### 3hree

In [123]:
# generate candidate triples
candidateThreePipe = frequentTwoPipe.join(frequentOnePipe).map(lambda x : x[-1]).map(splat).distinct().filter(doubled)
candidateThreePipe.take(5)

[('kyiv', 'russia', 'russian'),
 ('I', 'russia', 'russian'),
 ('russia', 'russian', 'war'),
 ('russia', 'russian', 'russiaukraineconflict'),
 ('russian', 'the', 'ukrain')]

In [124]:
# count three frequency
candidateThree = candidateThreePipe.collect() # pair in MEM
countThreePipe = dataPipe.map(lambda x : [(pair, is_sub(pair, x[-1])) for pair in candidateThree] ).flatMap(lambda x : x).reduceByKey(add)
countThreePipe.take(5)

[(('kyiv', 'russia', 'russian'), 4),
 (('I', 'russia', 'russian'), 15),
 (('russia', 'russian', 'war'), 8),
 (('russia', 'russian', 'russiaukraineconflict'), 5),
 (('russian', 'the', 'ukrain'), 33)]

In [125]:
# filter non frequent
frequentThreePipe = countThreePipe.filter(lambda x: x[-1] > THRESHOLD).map(lambda x: (1, x[0]))
frequentThreePipe.take(5)

[(1, ('russia', 'russian', 'ukrain'))]